In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))

In [ ]:
from vizro_ai import VizroAI
from langchain_anthropic import ChatAnthropic
import os
import vizro.plotly.express as px
from vizro_ai.plot2._response_models import ChartPlanStatic

In [ ]:
df = px.data.gapminder()

## Model

In [ ]:
# llm = ChatAnthropic(
#         model="claude-3-5-sonnet-20240620",
#         api_key = os.environ.get("ANTHROPIC_API_KEY"),
#         base_url= os.environ.get("ANTHROPIC_API_BASE")
#     )
llm = None
llm = "gpt-4o-mini-2024-07-18"

In [ ]:
vizro_ai = VizroAI(model=llm)

## Query

In [ ]:
query = "plot a bubble chart to shows the changes in life expectancy gdp per capita  over time. Animate the chart by year"
# query = "describe the composition of gdp in continent,and horizontal line for avg gdp"
# query = "show me the geo distribution of life expectancy and set year as animation "

## VizroAI

### Refactored version

In [ ]:
res2 = vizro_ai.plot2(df=df, user_input=query, return_elements=True)
res2.get_fig_object(data_frame=df)

In [ ]:
print(res2.code)

In [ ]:
print(res2.code_explanation)

In [ ]:
print(res2.chart_insights)

In [ ]:
# Underlying method for code properties
print(res2._get_complete_code(chart_name="other_figure",vizro=False))

In [ ]:
# Run chart with different data
res2.get_fig_object(data_frame=df.sample(200))

In [ ]:
# No code execution
res22 = vizro_ai.plot2(df=df, user_input=query, validate_code=False, return_elements=True)
print(res22.code_vizro)

### Old version

In [ ]:
res = vizro_ai.plot(df=df, user_input=query, explain=True, return_elements=True)
res.figure

# New version with Anthropic

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))

import os
# not all these 3 are needed. I just forgot which one is correct
print(os.environ.get("ANTHROPIC_API_URL"))
print(os.environ.get("ANTHROPIC_BASE_URL"))
print(os.environ.get("ANTHROPIC_API_BASE"))

from vizro_ai import VizroAI
vizro_ai = VizroAI(model="claude-3-sonnet-20240229")

vizro_ai.model

import vizro.plotly.express as px
query = "plot a bubble chart to shows the changes in life expectancy gdp per capita  over time. Animate the chart by year"
df = px.data.gapminder()

res2 = vizro_ai.plot2(df=df, user_input=query, return_elements=True)
res2.get_fig_object(data_frame=df)